# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124098e+02     1.670183e+00
 * time: 0.6052181720733643
     1     1.052503e+01     9.223864e-01
 * time: 1.6336572170257568
     2    -1.217641e+01     1.040792e+00
 * time: 2.143484115600586
     3    -3.423296e+01     7.737940e-01
 * time: 2.8672451972961426
     4    -4.773641e+01     5.649162e-01
 * time: 3.607746124267578
     5    -5.696506e+01     2.033505e-01
 * time: 4.319956064224243
     6    -5.964566e+01     2.029232e-01
 * time: 4.799715042114258
     7    -6.078294e+01     8.726615e-02
 * time: 5.309797048568726
     8    -6.133182e+01     4.307360e-02
 * time: 5.789692163467407
     9    -6.161459e+01     3.464900e-02
 * time: 6.269586086273193
    10    -6.181651e+01     2.791243e-02
 * time: 6.753457069396973
    11    -6.197237e+01     2.068270e-02
 * time: 7.2406980991363525
    12    -6.207241e+01     1.467854e-02
 * time: 7.7208411693573
    13    -6.212946e+01     1.262740e-02
 * time: 8.201327085494995
    1

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671052
    AtomicLocal         -18.8557660
    AtomicNonlocal      14.8522654
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485353 
    Xc                  -19.3336812

    total               -62.261666459099